# Using Schema Registry

In [1]:
# import json
# import uuid
# import os
# from dotenv import load_dotenv
# from pathlib import Path
# from faker import Faker
# from time import sleep
# from confluent_kafka import SerializingProducer
# from confluent_kafka.serialization import StringSerializer
# import random
# from confluent_kafka.schema_registry import SchemaRegistryClient
# import sys
# from confluent_kafka.schema_registry.protobuf import ProtobufSerializer


In [2]:
# Initialize Faker for fake data generation
# fake = Faker()

# # Function to create and serialize PetShopEvent data in JSON
# def create_petshop_event():
#     pet_event = {
#         "pet_id": str(uuid.uuid4()),
#         "pet_name": fake.first_name(),
#         "species": fake.random_element(elements=('Dog', 'Cat', 'Bird', 'Reptile')),
#         "breed": fake.random_element(elements=('Bulldog', 'Siamese', 'Parrot', 'Iguana')),
#         "age": fake.random_int(min=1, max=20),
#         "price": round(random.uniform(50.00, 5000.00), 2),
#         "weight": round(random.uniform(1.0, 100.0), 2),
#         "checkup_date": fake.date_this_decade().strftime('%Y-%m-%d')
#     }

#     # Print the generated PetShopEvent details
#     print(json.dumps(pet_event, indent=2))

# # Generate multiple fake events in a loop to test speed
# for _ in range(10):  # Generate 10 events for testing
#     create_petshop_event()

In [3]:
# # Load environment variables from .env file
# dotenv_path = Path('../.env')
# load_dotenv(dotenv_path=dotenv_path)

# kafka_host = os.getenv('KAFKA_HOST')
# topic_name = os.getenv('KAFKA_TOPIC_NAME')

# # Protobuf schema import
# sys.path.append('../protobuf')
# import schema_pb2

# # Kafka Configuration
# bootstrap_servers = f'{kafka_host}:9092'

# # Schema Registry Client
# schema_registry_client = SchemaRegistryClient({
#     'url': f'http://{os.getenv("SCHEMA_REG_HOST")}:8081'
# })

# # Protobuf Serializer
# protobuf_serializer = ProtobufSerializer(
#     schema_pb2.PetShopEvent, 
#     schema_registry_client, 
#     conf={'use.deprecated.format': False} 
# )

# # Producer Configuration (using protobuf_serializer for value)
# producer_conf = {
#     'bootstrap.servers': bootstrap_servers,
#     'key.serializer': StringSerializer('utf_8'),  
#     'value.serializer': protobuf_serializer,  
#     'message.timeout.ms': 10000
# }

# # Initialize Kafka producer
# producer = SerializingProducer(producer_conf)

# # Initialize Faker for fake data generation
# fake = Faker()


# def generate_pet_shop_event():
#     """Generate a fake PetShopEvent using Faker and Protobuf."""
#     pet_event = schema_pb2.PetShopEvent()
#     pet_event.pet_id = str(uuid.uuid4())
#     pet_event.pet_name = fake.first_name()
#     pet_event.species = fake.random_element(elements=('Dog', 'Cat', 'Bird', 'Reptile'))
#     pet_event.breed = fake.random_element(elements=('Bulldog', 'Siamese', 'Parrot', 'Iguana'))
#     pet_event.age = fake.random_int(min=1, max=20)
#     pet_event.price = round(random.uniform(50.00, 5000.00), 2)
#     pet_event.weight = round(random.uniform(1.0, 100.0), 2)
#     pet_event.checkup_date = fake.date_this_decade().strftime('%Y-%m-%d')  # Convert date to string
#     return pet_event

# # Generate and send one event every 5 seconds
# for i in range(1, 400):
#     pet_event = generate_pet_shop_event()
    
#     # Print the generated fake PetShopEvent (Protobuf object)
#     print(f"Generated PetShopEvent: {pet_event}")
    
#     # Send the event to Kafka topic
#     producer.produce(topic=topic_name, key=pet_event.pet_id, value=pet_event)
    
#     # Flush to ensure the message is sent
#     producer.flush()

#     # Wait before sending the next event
#     time.sleep(5)



# Without Schema Registry

In [4]:
import os
import sys
import uuid
import random
import time
from pathlib import Path
from faker import Faker
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import StringSerializer
from google.protobuf.message import EncodeError  
from dotenv import load_dotenv

# Import the generated Protobuf schema
sys.path.append('../protobuf')
import schema_pb2

# Load environment variables from .env file
dotenv_path = Path('../.env')
load_dotenv(dotenv_path=dotenv_path)

kafka_host = os.getenv('KAFKA_HOST')
topic_name = os.getenv('KAFKA_TOPIC_NAME')

# Kafka Configuration (No schema registry, using Protobuf directly)
bootstrap_servers = f'{kafka_host}:9092'

# Producer Configuration (Using Protobuf for value without Schema Registry)
producer_conf = {
    'bootstrap.servers': bootstrap_servers,
    'key.serializer': StringSerializer('utf_8'),  # Serializer for key
    'message.timeout.ms': 10000
}

# Initialize Kafka producer
producer = SerializingProducer(producer_conf)

# Initialize Faker for fake data generation
fake = Faker()

def generate_pet_shop_event():
    """Generate a fake PetShopEvent using Faker and Protobuf."""
    pet_event = schema_pb2.PetShopEvent()
    pet_event.pet_id = str(uuid.uuid4())
    pet_event.pet_name = fake.first_name()
    pet_event.species = fake.random_element(elements=('Dog', 'Cat', 'Bird', 'Reptile'))
    pet_event.breed = fake.random_element(elements=('Bulldog', 'Siamese', 'Parrot', 'Iguana'))
    pet_event.age = fake.random_int(min=1, max=20)
    pet_event.price = round(random.uniform(50.00, 5000.00), 2)
    pet_event.weight = round(random.uniform(1.0, 100.0), 2)
    pet_event.checkup_date = fake.date_this_decade().strftime('%Y-%m-%d')  # Convert date to string
    return pet_event

# Generate and send one event every 5 seconds
for i in range(1, 400):
    pet_event = generate_pet_shop_event()
    
    # Print the generated fake PetShopEvent (Protobuf object)
    print(f"Generated PetShopEvent: {pet_event}")
    
    try:
        # Serialize the Protobuf message to bytes
        pet_event_serialized = pet_event.SerializeToString()

        # Send the serialized event to Kafka topic
        producer.produce(topic=topic_name, key=pet_event.pet_id, value=pet_event_serialized)

        # Flush to ensure the message is sent
        producer.flush()

    except EncodeError as e:
        print(f"Error serializing Protobuf message: {e}")

    # Wait before sending the next event
    time.sleep(5)


Generated PetShopEvent: pet_id: "c466b21e-f9f1-4a32-a8bf-c9e36d6b40ee"
pet_name: "Edward"
species: "Reptile"
breed: "Iguana"
age: 7
price: 2749.2
weight: 40.44
checkup_date: "2023-08-29"

Generated PetShopEvent: pet_id: "0580879a-de0b-4a82-866f-51569b2afb2c"
pet_name: "Mackenzie"
species: "Cat"
breed: "Siamese"
age: 19
price: 4220.18
weight: 45.07
checkup_date: "2021-06-28"

Generated PetShopEvent: pet_id: "018e079e-8237-4ec0-bee4-d5bdb92e295c"
pet_name: "Brandi"
species: "Reptile"
breed: "Parrot"
age: 20
price: 78.23
weight: 36.62
checkup_date: "2024-07-01"

Generated PetShopEvent: pet_id: "2544dd16-58a9-4718-a882-4c6662f1ccfc"
pet_name: "Scott"
species: "Dog"
breed: "Iguana"
age: 8
price: 1145.18
weight: 56.31
checkup_date: "2020-10-13"

Generated PetShopEvent: pet_id: "3b67e105-a045-4ae7-8572-c3a1523303d0"
pet_name: "Samantha"
species: "Bird"
breed: "Iguana"
age: 9
price: 2832.33
weight: 81.81
checkup_date: "2020-11-03"

Generated PetShopEvent: pet_id: "d774a5a7-0509-41d7-afb4-a1ce1

KeyboardInterrupt: 